In [ ]:
!pip install igraph
!pip install scikit-image==0.20.0

In [5]:
from GNEMS.GNEMS import GNEMS_segment
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import numpy as np
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.color import label2rgb
from glob import glob
from tqdm import tqdm
import time
import threading
import json
import os

In [10]:
def run_tests(hyperparams, label=None):
    if label is None:
        method_name = f"GNEMS_alternate_{time.time()}"
    else:
        method_name = f"GNEMS_{label}"

    assert not os.path.exists(f"results/{method_name}"), "Directory already exists!"

    !mkdir results/{method_name}
    !mkdir results/{method_name}/noise
    !mkdir results/{method_name}/noise/0.5
    !mkdir results/{method_name}/noise/1.0
    !mkdir results/{method_name}/noise/2.0
    !mkdir results/{method_name}/noise/4.0
    !mkdir results/{method_name}/noise/8.0
    !mkdir results/{method_name}/clouds
    !mkdir results/{method_name}/texture

    with open(f"results/{method_name}/hyperparams.json", "w") as f:
        json.dump(hyperparams, f)

    def segment_and_save(path, id, bar=None):
        im = np.array(Image.open(path).convert("L"))
        d = hyperparams["d"]
        iterations = hyperparams["iterations"]
        prediction_stride = hyperparams["prediction_stride"]
        lambda_ = hyperparams["lambda_"]
        slic_segments = hyperparams["slic_segments"]
        sigma = hyperparams["sigma"]
        seg = GNEMS_segment(im, d=d, lambda_=lambda_, iterations=iterations, prediction_stride=prediction_stride, slic_segments=slic_segments, sigma=sigma, show_progress=False)
        category = path.split("/")[1]
        Image.fromarray(seg).convert("L").save(f"results/{method_name}/{category}/{id}.png")
        bar.update(1)

    print("Segmenting clouds dataset...")
    with open("datasets/clouds/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/clouds/images/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            while len(threading.enumerate()) >= 32:
                time.sleep(1)
            t.start()
            threads.append(t)
        for t in threads:
            t.join()

    with open("datasets/noise/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    print("Segmenting noise dataset...")
    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/noise/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            while len(threading.enumerate()) >= 32:
                time.sleep(1)
            t.start()
            threads.append(t)
        for t in threads:
            t.join()

    print("Segmenting texture dataset...")
    with open("datasets/texture/test_ids.txt", "r") as f:
        test_ids = [line.strip() for line in f.readlines()]

    threads = []
    with tqdm(total = len(test_ids)) as pbar:
        for id in test_ids:
            path = f"datasets/texture/images/{id}.png"
            t = threading.Thread(target=segment_and_save, args=(path,id,pbar))
            while len(threading.enumerate()) >= 32:
                time.sleep(1)
            t.start()
            threads.append(t)
        for t in threads:
            t.join()

In [9]:
hyperparams = {
    "d": 8,
    "iterations": 50,
    "prediction_stride": 8,
    "lambda_": 0.3,
    "n_filters": 16,
    "slic_segments": 100,
    "sigma": 3
}
run_tests(hyperparams, label="larger_patches")

mkdir: cannot create directory ‘results/GNEMS_larger_patches’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise/0.5’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise/1.0’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise/2.0’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise/4.0’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/noise/8.0’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/clouds’: File exists
mkdir: cannot create directory ‘results/GNEMS_larger_patches/texture’: File exists
Segmenting clouds dataset...


100%|██████████| 256/256 [00:52<00:00,  4.88it/s]


Segmenting noise dataset...


100%|██████████| 1280/1280 [04:21<00:00,  4.90it/s]


Segmenting texture dataset...


100%|██████████| 768/768 [02:39<00:00,  4.81it/s]
